***
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils

***
__landcover__:
_'discrete_classification'_ band of most recent image in _'COPERNICUS/Landcover/100m/Proba-V-C3/Global'_ collection
***

In [ ]:
#
# using discrete_classification band of most recent landcover image
#
eeclassification = (ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V-C3/Global')
                   .sort('system:time_start', False)  # reverse sort
                   .first()                           # most recent
                   .select('discrete_classification') # Land cover classification 0..200
                   .unmask(0, sameFootprint=False))   # 0 = Unknown; in gee used as mask
#
#
#
map = geemap.Map(height='600px')
map.centerObject(geeutils.bobspoint, 2)
map.addLayer(eeclassification) # the palette is in the image properties, but I wonder how the link is made.
map


***
__most frequent landuse class in an roi__ - using mode (unweighted)
***

In [ ]:
eepoint = geeutils.pixelinterspoint(geeutils.bobspoint, eeclassification)
#
# most frequent class in a region: reduceRegion(ee.Reducer.mode().unweighted())
# - in case we have the region congruent with the native raster 'unweighted()' is obsolete
# - for random regions we need unweighted() to avoid (float) values between two classes due to 'partial' of pixels
# - eventually "sea" (200) of "no data" (0) will always win for very large regions
#
map = geemap.Map(height='600px')
map.centerObject(eepoint, 4)
map.addLayer(eeclassification)
pixelsradius = 1
for iIdx in range(16):
    eeregion = geeutils.squarerasterboundsroi(eepoint, pixelsradius, eeclassification)
    map.addLayer(geeutils.outlinegeometryimage(eeregion, 5, 2), {'palette':'#000000'} )
    map.addLayer(geeutils.outlinegeometryimage(eeregion, 0, 2), {'palette':'#ff0000'} )
    # radius (half width - in pixels) of region 
    sz  = f"radius({pixelsradius})"
    # pixelcount in this roi
    sz += f" - pixels({eeclassification.reduceRegion(ee.Reducer.count(), eeregion, maxPixels=1e13).values().get(0).getInfo()})"
    # mode in this roi (most frequent pixel value - land use class)
    sz += f" - mode({eeclassification.reduceRegion(ee.Reducer.mode().unweighted(), eeregion, maxPixels=1e13).values().get(0).getInfo()})"
    # distinct pixel values (land use types) in this roi
    sz += f" - disctinct({eeclassification.reduceRegion(ee.Reducer.countDistinct(), eeregion, maxPixels=1e13).values().get(0).getInfo()})"
    print(sz)
    # start small and grow - distinct types should increase, eventually we'll expect mostly water (class 200)
    pixelsradius = pixelsradius * 2
map


***
some properties
- one image per year 2015-2019
- Web-Mercator 100m
- classes 0-200
***

In [ ]:
eeimagecollection = ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V-C3/Global')
eeimagecollection = eeimagecollection.select('discrete_classification')
#
# some properties
#
print(geeutils.szimagecollectioninfo(eeimagecollection))
#
# EPSG:3857 Pseudo-Mercator aka Web-Mercator
#
print(geeutils.szprojectioninfo(eeimagecollection.first()))
#
# unbound 
#
print(f"unbound ? {eeimagecollection.first().geometry().isUnbounded().getInfo()}")
#
# Image.reduceRegion does not work on unbounded images
#      print(geeutils.szestimatevaluesinfo(eeimagecollection.first() - unbound
#
# clipping to a region?
#    https://groups.google.com/g/google-earth-engine-developers/c/eO22be7vsPw/m/nJxM2QVEAQAJ
#    Noel Gorelick - 6 October 2016
#    We have problems with global rectangles; 
#    best to just avoid them and use a non-geodesic polygon with more than 4 points instead:
#    region = ee.Geometry.Polygon([-180, 90, 0, 90, 180, 90, 180, -90, 10, -90, -180, -90], null, false)
#
#      region = ee.Geometry.Polygon([-180, 90, 0, 90, 180, 90, 180, -90, 10, -90, -180, -90], None, False)
#      print(geeutils.szestimatevaluesinfo(eeimagecollection.first().clip(region))) - still unbound
# 
# there seems to be no way to obtain a global roi.
# there seems to be no way to obtain a 'nearly' global roi neither
# and I cannot find any guide, spec, explanation, ... telling me anything about the criteria or limitations
#
#      region = ee.Geometry.Polygon([-150, 80,        150, 80, 150, -80,        -150, -80, -150, 80], None, False)
#      print(geeutils.szestimatevaluesinfo(eeimagecollection.first().clip(region))) - still unbound but '149' becomes bound
#
region_geodesic    = ee.Geometry.Polygon([-149, 80,        149, 80, 149, -80,        -149, -80, -149, 80], None, False)
region_nongeodesic = ee.Geometry.Polygon([-149, 80, 0, 80, 149, 80, 149, -80, 0, -80,-149, -80, -149, 80], None, True)
print(geeutils.szestimatevaluesinfo(eeimagecollection.first().clip(region_geodesic)))
print(geeutils.szestimatevaluesinfo(eeimagecollection.first().clip(region_nongeodesic)))

map = geemap.Map(height='600px')
map.centerObject(geeutils.bobspoint, 0)
map.addLayer(geeutils.outlinegeometryimage(region_nongeodesic, 0, 10))
map

In [ ]:
#
# apparently gee can't make up its own mind at +/- 180 longitude and +/- 90 latitude
#
import math
map = geemap.Map(height='600px')
eeimage = ee.Image.pixelLonLat()
print(geeutils.szprojectioninfo(eeimage))
map.centerObject(ee.Geometry.Point(0, 0), 0)
map.addLayer(eeimage.select('longitude'), {'min':-180, 'max':180})
map.addLayer(eeimage.select('latitude'),  {'min':-90,  'max':90})
for pointlon in range (-180,181,60):
    for pointlat in range (-90,91,30):
        eepoint  =  ee.Geometry.Point(pointlon, pointlat)
        eeregion = geeutils.squarerasterboundsroi(eepoint, 0.6, eeimage)
        eeregion = geeutils.squareareaboundsroi(eepoint, 111320)
        print(math.sqrt(eeregion.area(maxError=0.001).getInfo()), geeutils.szgeometryinfo(eeregion.centroid(maxError=0.001)) )
        print(geeutils.szestimatevaluesinfo(eeimage.clip(eeregion)))
        map.addLayer(geeutils.outlinegeometryimage(eeregion, 100000, 10))
map

***
__random points__ ignoring sea
***

In [ ]:
import random

#
# using discrete_classification band of most recent landcover image
#
eeclassification = (ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V-C3/Global')
                   .sort('system:time_start', False)  # reverse sort
                   .first()                           # most recent
                   .select('discrete_classification') # Land cover classification 0..200
                   .unmask(0, sameFootprint=False))   # 0 = Unknown; in gee used as mask

#
# assume we skip regions with mainly
#     0 (Unknown. No or not enough satellite data available.)
#    70 (Snow and ice. Lands under snow or ice cover throughout the year.)
#    80 (Permanent water bodies. Lakes, reservoirs, and rivers. Can be either fresh or salt-water bodies.)
#   200 (Oceans, seas. Can be either fresh or salt-water bodies.)
#
lstskiplanduseclasses = [0, 70, 80, 200]


lstfeatures = []
cPatch = 0
iPatch = 0
while True:
    iPatch += 1
    #
    # assume global samples
    #
    longitude = (1 - random.random())*360. - 180.
    latitude  = random.uniform(-80,80)
    eepoint   = ee.Geometry.Point(longitude, latitude)
    #
    # assume landuse determined by about 1km x 1 km environment
    #
    landuseclass = eeclassification.reduceRegion(
        ee.Reducer.mode().unweighted(), eepoint.buffer(500, maxError=0.001)).values().get(0).getInfo()
    #
    # skip specified land use classes
    #
    if landuseclass in lstskiplanduseclasses:
        print(f"({iPatch:5d}) - skipping patch at ({longitude:013.8f}, {latitude:013.8f}): class({landuseclass:3d}) ")
        continue

    cPatch += 1
    print(f"({iPatch:5d}) - patch at ({longitude:013.8f}, {latitude:013.8f}): class({landuseclass:3d}) patches:{cPatch:5d}")

    eeregion = geeutils.squarerasterboundsroi(geeutils.pixelinterspoint(eepoint, eeclassification), 1, eeclassification)
    #print(geeutils.szgeometryinfo(eeregion))
    
    eefeature = ee.Feature(eeregion, {'lon': longitude, 'lat': latitude, 'class': landuseclass})
    lstfeatures.append(eefeature)

    if cPatch > 1000:
        break

eeFeatureCollection = ee.FeatureCollection(lstfeatures)
geemap.ee_export_vector(eeFeatureCollection, r"C:\tmp\tst.shp")
